In [1]:
# Student Number: C1893753

# Part 3 - Text analysis and ethics

# 3.a Computing PMI

In this assessment you are tasked to discover strong associations between concepts in Airbnb reviews. The starter code we provide in this notebook is for orientation only. The below imports are enough to implement a valid answer.

### Imports, data loading and helper functions

We first connect our google drive, import pandas, numpy and some useful nltk and collections modules, then load the dataframe and define a function for printing the current time, useful to log our progress in some of the tasks.

In [44]:
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [45]:
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [46]:
# load stopwords
sw = set(stopwords.words('english'))

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
p = '/content/drive/MyDrive/'
df = pd.read_csv(os.path.join(p,'reviews.csv'))
# dealing with empty reviews
df.comments = df.comments.fillna('')

In [49]:
df.head()

listing_id  ...                                           comments
0        2818  ...  Daniel is really cool. The place was nice and ...
1        2818  ...  Daniel is the most amazing host! His place is ...
2        2818  ...  We had such a great time in Amsterdam. Daniel ...
3        2818  ...  Very professional operation. Room is very clea...
4        2818  ...  Daniel is highly recommended.  He provided all...

[5 rows x 6 columns]

In [50]:
df.shape

(452143, 6)

### 3.a1 - Process reviews

What to implement: A `function process_reviews(df)` that will take as input the original dataframe and will return it with three additional columns: `tokenized`, `tagged` and `lower_tagged`.

In [51]:
def process_reviews(df):
  df = df[:20000] # can change for testing as dataset is very large
  comments = df["comments"].to_list() # comments to list
  # tokenized
  tokenized = [word_tokenize(comment) for comment in comments] # converting to word tokenization
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^\n\t&*_~'''
  remove_punct_inTokenized = [[token for token in tokens if token not in punctuations] for tokens in tokenized] # removing the punctuations
  # tagged
  tagged = [pos_tag(tokens) for tokens in remove_punct_inTokenized] # applying parts of speech tagging
  lemma = WordNetLemmatizer()
  # removing the stop words, lower case and applying lemmatization to reduce vocabulary
  lemmatized = [[lemma.lemmatize(token.lower()) for token in tokens if token.lower() not in sw] for tokens in remove_punct_inTokenized]
  # lower_tagged
  lower_tagged = [pos_tag(tokens) for tokens in lemmatized] # applying parts of speech tagging
  # Adding columns in dataframe
  df["tokenized"] = tokenized
  df["tagged"] = tagged
  df["lower_tagged"] = lower_tagged
  return df

In [52]:
df = process_reviews(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

### 3.a2 - Create a vocabulary

What to implement: A function `get_vocab(df)` which takes as input the DataFrame generated in step 1.c, and returns two lists, one for the 1,000 most frequent center words (nouns) and one for the 1,000 most frequent context words (either verbs or adjectives). 

In [53]:
def get_vocab(df):
  cent_vocab = []
  cont_vocab = []
  # lower_tagged to list
  lower_tagged = df['lower_tagged'].to_list()
  cent_vocab_dic = {}
  cont_vocab_dic = {}
  # Finding Noun, Verb and Adjective and calculating frequency
  for tags in lower_tagged:
     for tag in tags:
      # checking noun
        if tag[-1][0] == "N":
        #calculating frequency
          if tag[0] in cent_vocab_dic:
            cent_vocab_dic[tag[0]] += 1
          else:
            cent_vocab_dic[tag[0]] = 1
          #checking Verb and Adjective
        elif tag[-1][0] == "J" or tag[-1][0] == "V":
          #calculating frequency
          if tag[0] in cont_vocab_dic:
            cont_vocab_dic[tag[0]] += 1
          else:
            cont_vocab_dic[tag[0]] = 1
  # sorting dictionary cent_vocab_dic on frequnecy as value in ascending order and returning words as key in list
  cent_vocab = sorted(cent_vocab_dic, key=cent_vocab_dic.get, reverse=True)
  # sorting dictionary cont_vocab_dic on frequnecy as value in ascending order and returning words as key in list
  cont_vocab = sorted(cont_vocab_dic, key=cont_vocab_dic.get, reverse=True)
  # returning 2 lists of only 1000 most frequent words 
  return cent_vocab[:1000], cont_vocab[:1000]

In [54]:
cent_vocab, cont_vocab = get_vocab(df)

In [55]:
len(cont_vocab)

1000

### 3.a3 Count co-occurrences between center and context words

What to implement: A function `get_coocs(df, center_vocab, context_vocab)` which takes as input the DataFrame generated in step 1, and the lists generated in step 2 and returns a dictionary of dictionaries, of the form in the example above. It is up to you how you define context (full review? per sentence? a sliding window of fixed size?), and how to deal with exceptional cases (center words occurring more than once, center and context words being part of your vocabulary because they are frequent both as a noun and as a verb, etc). Use comments in your code to justify your approach. 

In [14]:
def get_coocs(df, cent_vocab, cont_vocab):
    coocs = {}
    # dataframe comments into list as reviews
    reviews = df["comments"].to_list()
    # loop in cent_vocab
    for cent in cent_vocab:
        # context coocs dictionary
        cont_coocs = {}
        # loop in reviews
        for review in reviews:
            # checking the center vocab is whether in review
            if cent in review.lower():
                # loop in cont_vocab
                for cont in cont_vocab:
                    # checking the context vocab is whether in review
                    if cont in review.lower():
                        # making a dictionary occurrence for context vocab
                        if cont in cont_coocs:
                            cont_coocs[cont] += 1
                        else:
                            cont_coocs[cont] = 1
                    else:
                        cont_coocs[cont] = 0
        # storing the center key with value as context dictionry
        coocs[cent] = cont_coocs
    return coocs  

In [56]:
coocs = get_coocs(df, cent_vocab, cont_vocab)

In [59]:
len(coocs)

1000

In [61]:
len(cont_vocab)

1000

In [62]:
len(cent_vocab)

1000

### 3.a4 Convert co-occurrence dictionary to 1000x1000 dataframe
What to implement: A function called `cooc_dict2df(cooc_dict)`, which takes as input the dictionary of dictionaries generated in step 3 and returns a DataFrame where each row corresponds to one center word, and each column corresponds to one context word, and cells are their corresponding co-occurrence value. Some (x,y) pairs will never co-occur, you should have a 0 value for those cases. 

In [57]:
def cooc_dict2df(coocs):
  coocdf=pd.DataFrame.from_dict(coocs)
  return coocdf

In [58]:
coocdf = cooc_dict2df(coocs)
coocdf.shape

(1000, 1000)

### 3.a5 Raw co-occurrences to PMI scores

What to implement: A function `cooc2pmi(df)` that takes as input the DataFrame generated in step 4, and returns a new DataFrame with the same rows and columns, but with PMI scores instead of raw co-occurrence counts. 

In [63]:
def cooc2pmi(df):
  row_totals = df.sum(axis=1).astype(float)
  prob_cols_given_row = (df.T / row_totals).T

  total_columns = df.sum(axis=0).astype(float)
  probability_columns = total_columns / sum(total_columns)

  ratio = prob_cols_given_row / probability_columns
  ratio[ratio==0] = 0.00001
  pmidf = np.log(ratio)
  pmidf[pmidf < 0] = 0
  return pmidf

In [64]:
pmidf = cooc2pmi(coocdf)
pmidf.shape

(1000, 1000)

In [65]:
pmidf

place  apartment  location  ...    making    comodo    stanza
great        0.0        0.0       0.0  ...  2.089455  0.000000  0.000000
u            0.0        0.0       0.0  ...  0.935869  0.575345  0.157953
nice         0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
clean        0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
recommend    0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
...          ...        ...       ...  ...       ...       ...       ...
riding       0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
airy         0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
offering     0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
hang         0.0        0.0       0.0  ...  0.000000  0.000000  0.000000
été          0.0        0.0       0.0  ...  0.000000  0.000000  0.000000

[1000 rows x 1000 columns]

### 3.a6 Retrieve top-k context words, given a center word

What to implement: A function `topk(df, center_word, N=10)` that takes as input: (1) the DataFrame generated in step 5, (2) a `center_word` (a string like `‘towels’`), and (3) an optional named argument called `N` with default value of 10; and returns a list of `N` strings, in order of their PMI score with the `center_word`. You do not need to handle cases for which the word `center_word` is not found in `df`. 

In [66]:
def topk(df, center_word, N=10):
  top_words = df[center_word].sort_values(ascending = False).head(10)
  return top_words

In [67]:
topk(pmidf, 'restaurant')

restaurant    5.175915
rest          4.292564
est           3.583467
au            3.230128
tiny          1.651579
nearby        1.651579
next          1.651579
special       1.119362
u             0.772598
sink          0.677530
Name: restaurant, dtype: float64

# 3.b Ethical, social and legal implications



Local authorities in touristic hotspots like Amsterdam, NYC or Barcelona regulate the price of recreational apartments for rent to, among others, ensure that fair rent prices are kept for year-long residents. Consider your price recommender for hosts in Question 2c. Imagine that Airbnb recommends a new host to put the price of your flat at a price which is above the official regulations established by the local government. Upon inspection, you realize that the inflated price you have been recommended comes from many apartments in the area only being offered during an annual event which brings many tourists, and which causes prices to rise. 

In this context, critically reflect on the compliance of this recommender system with **one of the five actions** outlined in the **UK’s Data Ethics Framework**. You should prioritize the action that, in your opinion, is the weakest. Then, justify your choice by critically analyzing the three **key principles** outlined in the Framework, namely _transparency_, _accountability_ and _fairness_. Finally, you should propose and critically justify a solution that would improve the recommender system in at least one of these principles. You are strongly encouraged to follow a scholarly approach, e.g., with peer-reviewed references as support. 

Your report should be between 500 and 750 words long.  

### Your answer here. No Python, only Markdown.

Write your answer after the line.

---

...[The Data Ethics Framework, i.e., some set of principles are used to guide government and public sector organizations to use public data in accurate way. This is required to plan, implement and evaluate data for new policies or services. 
It appears from the context that the recommender system doesnot comply with the law as a little clarity has been provided regarding legal requirements for this project.
Data Ethics Framework consists of three key principles. They are as follows:
(i) Transparency: Transparency of the data means it should be open, complete, easily understandable and accessible. Our data shows all the required aspects very clearly, which satisfies the conditions for transparency. But, according to the context, we can comment that the legal criteria for the given project has to be a little more clear. 
(ii) Accountability: A project should have a governance and oversight mechanism. The public and their representatives can effectively oversee and control the decisions and actions taken by government officials, so the project can accomplish its goals. However, being able to demonstrate what steps we're taking to ensure documentation is a vital part of complying with data protection laws. Also, this will be the responsibility of the recommender system, which will ensure policies and training are in place. Detailed documentation of things like Data Protection Impact Assessments is also essential to demonstrate how the system operates on an individual basis. 
(iii) Fairness: We have to ensure that no discriminatory effects are unintentionally created and minimize biases that may influence the outcomes of our model. Furthermore, we must ensure that the project's outcomes are unbiased, non-discriminatory, and consistent with democratic values, including human rights.
Accordingly, the Airbnb recommended system of accountability is characterized by effective project governance, which signifies that the bills charged to tourists are acceptable because they are the result of research that was not only conducted, but also examined. Despite the fact that the recommended solution respects the Data Ethics Framework's fairness criterion, the data protection law must be evident to all. 
Rather than relying on the number of bookings at a particular time, the program relies on other key factors that reflect the interests of key stakeholders, such as the host and customers. Due to the project's fairness for people from all backgrounds and social groups, no one will feel discriminated, as it will not take into account irrelevant data that may indicate a person's social status or grouping. Transparency implies open access to all data and processes, in understandable, accessible formats that are free, and guidance and assistance will be provided for all users. As a consequence, it complies with the data ethics framework in the UK.
But at the end, we can comment on the recommender system that it has to be more tranparent about the data protection law to the customers. It hasn't been established in correct manner how to scrutinize, govern, or peer review the recommender system for data protection law. It is very likely that the recommender will cause harm to customers or have detrimental and discriminatory effects.]...